In [1]:
import pandas as pd

vuelos = pd.read_csv("flights.csv")

C:\Users\carlo\AppData\Local\Temp\ipykernel_29816\3486561847.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  vuelos = pd.read_csv("flights.csv")


In [10]:
# Extraer un datagrame reducido en filas con el 0,1% de los datos random

vuelos_reducido = vuelos.sample(frac=0.05, random_state=1)

display(vuelos_reducido)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4605645,2015,10,14,3,UA,356,N494UA,11618,13930,1945,...,2104.0,-20.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
398028,2015,1,27,2,MQ,3458,N671MQ,ORD,MLI,820,...,1100.0,107.0,0,0,NaN,34.0,0.0,0.0,63.0,10.0
2978311,2015,7,6,1,B6,1202,N535JB,FLL,JFK,1507,...,1809.0,-4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2602363,2015,6,13,6,DL,2529,N301DQ,ATL,CLE,2204,...,2343.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2478083,2015,6,6,6,WN,2560,N7729A,CLT,BWI,1325,...,1459.0,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763374,2015,2,20,5,DL,1472,N540US,ATL,TPA,815,...,928.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
885934,2015,2,27,5,WN,130,N205WN,SLC,DEN,2200,...,44.0,84.0,0,0,NaN,0.0,0.0,0.0,84.0,0.0
808181,2015,2,23,1,DL,1403,N987AT,TYS,ATL,830,...,949.0,9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3389527,2015,7,30,4,AA,1754,N710UW,CLT,BDL,1849,...,2036.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# guardar el dataframe reducido en un nuevo archivo CSV
vuelos_reducido.to_csv("flights_reducido.csv", index=False)

In [13]:
import numpy as np

# Cargar datos
flights = pd.read_csv('flights_reducido.csv')
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')

# 1. CREAR COLUMNA DE FECHA
flights['DATE'] = pd.to_datetime(flights[['YEAR', 'MONTH', 'DAY']])

# 2. FUNCIÓN PARA CONVERTIR FORMATO HHMM A CADENA HH:MM
def format_time(x):
    if pd.isnull(x):
        return np.nan
    if x == 2400: # Manejar caso borde de medianoche
        return '00:00'
    x = int(x)
    return f"{x // 100:02d}:{x % 100:02d}"

# Aplicar a columnas de tiempo clave
time_cols = ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME']
for col in time_cols:
    flights[col + '_FORMATTED'] = flights[col].apply(format_time)

# 3. MANEJO DE VALORES NULOS
# Rellenar causas de retraso con 0 (asumimos que si es nulo, no hubo ese tipo de retraso)
delay_cols = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
flights[delay_cols] = flights[delay_cols].fillna(0)

# Opcional: Eliminar vuelos cancelados para análisis de retrasos
flights_cleaned = flights # esta linea hace que no se eliminen los vuelos cancelados, sino añadir #flights_cleaned = flights[flights['CANCELLED'] == 0]

# 4. LIMPIEZA DE CÓDIGOS DE AEROPUERTO (Eliminar numéricos si no se pueden mapear)
# Mantenemos solo los que tienen longitud 3 (IATA codes)
flights_cleaned = flights_cleaned[flights_cleaned['ORIGIN_AIRPORT'].apply(lambda x: len(str(x)) == 3)]
flights_cleaned = flights_cleaned[flights_cleaned['DESTINATION_AIRPORT'].apply(lambda x: len(str(x)) == 3)]

# 5. MERGE CON AEROLÍNEAS Y AEROPUERTOS
flights_final = flights_cleaned.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')
# Renombrar columna para evitar duplicados
flights_final = flights_final.rename(columns={'AIRLINE_y': 'AIRLINE_NAME', 'AIRLINE_x': 'AIRLINE_CODE'})
flights_final = flights_final.drop('IATA_CODE', axis=1)

print("Preprocessing completado.")
display(flights_final)

Preprocessing completado.


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,SCHEDULED_DEPARTURE_FORMATTED,DEPARTURE_TIME_FORMATTED,SCHEDULED_ARRIVAL_FORMATTED,ARRIVAL_TIME_FORMATTED,AIRLINE_NAME
0,2015,1,27,2,MQ,3458,N671MQ,ORD,MLI,820,...,0.0,0.0,63.0,10.0,2015-01-27,08:20,09:33,09:13,11:00,American Eagle Airlines Inc.
1,2015,7,6,1,B6,1202,N535JB,FLL,JFK,1507,...,0.0,0.0,0.0,0.0,2015-07-06,15:07,15:22,18:13,18:09,JetBlue Airways
2,2015,6,13,6,DL,2529,N301DQ,ATL,CLE,2204,...,0.0,0.0,0.0,0.0,2015-06-13,22:04,22:04,23:52,23:43,Delta Air Lines Inc.
3,2015,6,6,6,WN,2560,N7729A,CLT,BWI,1325,...,0.0,0.0,0.0,0.0,2015-06-06,13:25,13:35,15:00,14:59,Southwest Airlines Co.
4,2015,3,21,6,WN,3550,N914WN,MDW,BOS,1950,...,0.0,6.0,14.0,0.0,2015-03-21,19:50,20:22,23:00,23:20,Southwest Airlines Co.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266588,2015,2,20,5,DL,1472,N540US,ATL,TPA,815,...,0.0,0.0,0.0,0.0,2015-02-20,08:15,08:10,09:35,09:28,Delta Air Lines Inc.
266589,2015,2,27,5,WN,130,N205WN,SLC,DEN,2200,...,0.0,0.0,84.0,0.0,2015-02-27,22:00,23:34,23:20,00:44,Southwest Airlines Co.
266590,2015,2,23,1,DL,1403,N987AT,TYS,ATL,830,...,0.0,0.0,0.0,0.0,2015-02-23,08:30,08:23,09:40,09:49,Delta Air Lines Inc.
266591,2015,7,30,4,AA,1754,N710UW,CLT,BDL,1849,...,0.0,0.0,0.0,0.0,2015-07-30,18:49,18:51,20:46,20:36,American Airlines Inc.
